In [1]:
import numpy as np
import keras
np.random.seed(123)  
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import glob
from matplotlib import pyplot as plt


%matplotlib inline
from keras.datasets import mnist

batch_size = 10
num_classes = 10
epochs = 20

image_generator = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        data_format="channels_last",         
        horizontal_flip=True)



def batch_process(dir):
        return  image_generator.flow_from_directory(
        dir,  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 28X28
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

def train_generator():    
    return batch_process('images/training')  
def validation_generator():
    return batch_process('images/testing')

 
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D
def get_model(retrain):
           
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=( 28, 28,1),data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

def process_model(retrain):
    model=get_model(retrain);
    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    
    model.fit_generator(
        train_generator(),
        steps_per_epoch=500 // batch_size,
        epochs=epochs,
        validation_data=validation_generator()
        )
    
    
    model.save('first_try.h5')
    
def main():
     process_model(False)
    
     
    
if __name__ == "__main__":
    main()    

e:\tettu\python364\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 22 images belonging to 2 classes.
Found 4 images belonging to 2 classes.
Epoch 1/20
50/50 [==============================] - 6s 130ms/step - loss: 0.6966 - acc: 0.4984 - val_loss: 0.6883 - val_acc: 0.5000
Epoch 2/20
50/50 [==============================] - 2s 36ms/step - loss: 0.6671 - acc: 0.6455 - val_loss: 0.5640 - val_acc: 1.0000
Epoch 3/20
50/50 [==============================] - 2s 33ms/step - loss: 0.5845 - acc: 0.7161 - val_loss: 0.3875 - val_acc: 1.0000
Epoch 4/20
50/50 [==============================] - 2s 34ms/step - loss: 0.4449 - acc: 0.8200 - val_loss: 0.2947 - val_acc: 1.0000
Epoch 5/20
50/50 [==============================] - 2s 35ms/step - loss: 0.3027 - acc: 0.8919 - val_loss: 0.1702 - val_acc: 1.0000
Epoch 6/20
50/50 [==============================] - 2s 36ms/step - loss: 0.2435 - acc: 0.9438 - val_loss: 0.1070 - val_acc: 1.0000
Epoch 7/20
50/50 [==============================] - 2s 35ms/step - loss: 0.1638 - acc: 0.9698 - val_loss: 0.4805 - val_acc: 0.7500
Epo